### Target Webscraping

Using Selenium again to go through all the data in Target and see what we can get

Source list
+ https://www.youtube.com/watch?v=SPM1tm2ZdK4&t=855s
+ https://www.youtube.com/watch?v=USrjHgO9Niw python selenium button clicking


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import csv
import re
import pandas as pd 

grocery_list = pd.read_csv('Grocery List.csv')

college_file_list = ['ames, ia', 'college station, tx', 'minneapolis, mn'] 
# with open('Collegetowns2.csv', 'r') as file:
#     # Create a CSV reader object using DictReader
#     for line in file:
#         city = line[:-1].replace(',', '-').replace(' ', '-')
#         college_file_list.append(city)

food_prices = pd.DataFrame()

options=Options()
options.add_experimental_option("detach", True)

In [13]:
food_prices

Item,Ground Beef x 2,1/2 gallon milk,Cereal,Lunch meat,Bread,American cheese,Ramen pack,Poptarts,Frozen pizza,Eggs Dozen,...,Crushed Tomatoes,Light Red Kidney Beans,Spaghetti Noodles,Spaghetti Sauce,Green Peppers,Tomato Paste,Onions x2,Apple Juice,Butter 1/4,Coffee 1/4
URL,https://www.target.com/p/all-natural-85-15-gro...,https://www.target.com/p/2-reduced-fat-milk-0-...,https://www.target.com/p/cheerios-breakfast-ce...,https://www.target.com/p/uncured-honey-ham-ult...,https://www.target.com/p/sara-lee-whole-grain-...,https://www.target.com/p/kraft-singles-america...,https://www.target.com/p/maruchan-souper-6-pac...,https://www.target.com/p/pop-tarts-frosted-bro...,https://www.target.com/p/jack-39-s-original-th...,https://www.target.com/p/grade-a-large-eggs-12...,...,https://www.target.com/p/muir-glen-organic-glu...,https://www.target.com/p/organic-low-sodium-li...,https://www.target.com/p/barilla-spaghetti-pas...,https://www.target.com/p/tomato-basil-38-garli...,https://www.target.com/p/green-bell-peppers-2c...,https://www.target.com/p/tomato-paste-6oz-good...,https://www.target.com/p/yellow-onion-each/-/A...,https://www.target.com/p/100-apple-juice-64-fl...,https://www.target.com/p/land-o-lakes-unsalted...,https://www.target.com/p/folgers-classic-mediu...


In [28]:
# grocery_list.T.iloc[0]
grocery_df = grocery_list.T
grocery_df.columns = grocery_df.iloc[0]
grocery_df = grocery_df[1:]
grocery_df = grocery_df.reset_index(drop=True)
grocery_df.columns

food_prices = pd.DataFrame(college_file_list, columns=["Location"])
food_prices = pd.concat([food_prices, grocery_df], axis=1)
food_prices

,Location,Dark Red Kidney Beans,Ground Beef x 2,1/2 gallon milk,Cereal,Lunch meat,Bread,American cheese,Ramen pack,Poptarts,...,Crushed Tomatoes,Light Red Kidney Beans,Spaghetti Noodles,Spaghetti Sauce,Green Peppers,Tomato Paste,Onions x2,Apple Juice,Butter 1/4,Coffee 1/4
0,"ames, ia",https://www.target.com/p/bush-39-s-dark-red-ki...,https://www.target.com/p/all-natural-85-15-gro...,https://www.target.com/p/2-reduced-fat-milk-0-...,https://www.target.com/p/cheerios-breakfast-ce...,https://www.target.com/p/uncured-honey-ham-ult...,https://www.target.com/p/sara-lee-whole-grain-...,https://www.target.com/p/kraft-singles-america...,https://www.target.com/p/maruchan-souper-6-pac...,https://www.target.com/p/pop-tarts-frosted-bro...,...,https://www.target.com/p/muir-glen-organic-glu...,https://www.target.com/p/organic-low-sodium-li...,https://www.target.com/p/barilla-spaghetti-pas...,https://www.target.com/p/tomato-basil-38-garli...,https://www.target.com/p/green-bell-peppers-2c...,https://www.target.com/p/tomato-paste-6oz-good...,https://www.target.com/p/yellow-onion-each/-/A...,https://www.target.com/p/100-apple-juice-64-fl...,https://www.target.com/p/land-o-lakes-unsalted...,https://www.target.com/p/folgers-classic-mediu...
1,"college station, tx",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"minneapolis, mn",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
food_prices = pd.DataFrame(college_file_list, columns=["Location"])
food_prices

,Location
0,"ames, ia"
1,"college station, tx"
2,"minneapolis, mn"


In [8]:
for index, row in grocery_list.iterrows():
    print("Item:", row['Item'])
    print("URL:", row['URL'])

Item: Dark Red Kidney Beans
URL: https://www.target.com/p/bush-39-s-dark-red-kidney-beans-16oz/-/A-13593955#lnk
Item: Ground Beef x 2
URL: https://www.target.com/p/all-natural-85-15-ground-beef-1lb-good-38-gather-8482/-/A-13288295#lnk
Item: 1/2 gallon milk
URL: https://www.target.com/p/2-reduced-fat-milk-0-5gal-good-38-gather-8482/-/A-13276128#lnk
Item: Cereal
URL: https://www.target.com/p/cheerios-breakfast-cereal/-/A-88752573?preselect=81875238#lnk
Item: Lunch meat
URL: https://www.target.com/p/uncured-honey-ham-ultra-thin-deli-slices-9oz-good-38-gather-8482/-/A-54528288#lnk
Item: Bread
URL: https://www.target.com/p/sara-lee-whole-grain-soft-white-bread-20oz/-/A-13376577#lnk
Item: American cheese
URL: https://www.target.com/p/kraft-singles-american-cheese-slices-12oz-16ct/-/A-12955140#lnk
Item: Ramen pack
URL: https://www.target.com/p/maruchan-souper-6-pack-chicken-ramen-noodle-soup-18oz-6ct/-/A-14770997#lnk
Item: Poptarts
URL: https://www.target.com/p/pop-tarts-frosted-brown-sugar-c

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                            options=options)

for index, row in grocery_list.iterrows():
    print("Starting on: ", row["Item"])
    for college in college_file_list:

        # Dark red kidney beans ames iowa
        driver.get(row["URL"])

        # Find the element by data-test attribute
        price_element = driver.find_element(By.CSS_SELECTOR, '[data-test="product-price"]')
        print(price_element.text)

        time.sleep(2)

        # Find the button by its ID
        webStoreButton = driver.find_element(By.ID, "web-store-id-msg-btn")

        # Click the button
        webStoreButton.click()
        time.sleep(2)

        inputBox = driver.find_element(By.ID, "zip-or-city-state")
        inputBox.send_keys(college) # Need to input "city, st"

        lookupButton =  driver.find_element(By.CSS_SELECTOR, '[data-test="@web/StoreLocationSearch/button"]')
        lookupButton.click()
        time.sleep(2)

        ShopStoreButton =  driver.find_element(By.CSS_SELECTOR, '[data-test="@web/StoreMenu/ShopThisStoreButton"]')
        ShopStoreButton.click()

        time.sleep(2)
        # ID of the button id="zip-code-id-btn"


driver.close()

Starting on:  Dark Red Kidney Beans
$1.29
$1.29
$1.29
Starting on:  Ground Beef
$6.49
$6.49
$6.49
